In [4]:
import pandas as pd 
import requests
import os 
from dotenv import load_dotenv, find_dotenv
import pickle
load_dotenv(find_dotenv())

True

In [8]:
with open('test.pickle', 'rb') as file:
    test = pickle.load(file)

In [11]:
df = pd.DataFrame(test)

In [14]:
df[df['ouid'].notnull()]

,ouid,error
0,9dee65f46e2ab0b7890f5e7faca4e2d2,NaN
1,c2184938c04475eb1176a662c7bd4f20,NaN
2,27a2efa68d0ff3a676cbb67022e8f88b,NaN
3,79428c08c2530a754d27686cb29abadc,NaN
4,ca39161ac838183530d4a1248907b71b,NaN
...,...,...
4995,8fe8dc451fea2c0d546d655f3d3adb33,NaN
4996,e36e8cff633572467a56479d71c6bf53,NaN
4997,81a3529e0273aeca11867e1c17b964bc,NaN
4998,f87529caaf63c5111e30ff1e3a511c4b,NaN
